In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt;
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import NearestCentroid

import time
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import randint
import natsort
# Importing Keras libraries
#from keras.utils import np_utils              It was older statement. Gave error. Changed by below. In below code need to change np_utils by utils
from keras import utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
#from keras.preprocessing.image import load_img
from tensorflow.keras.utils import load_img
#from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import itertools
from sklearn.model_selection import KFold
import numpy as np
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.plots import plot_fitness_evolution, plot_search_space
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import KFold
import numpy

import matplotlib.pyplot as plt

from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score

import math

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

In [3]:
#Loading Data of Main Database

X_H_All = [os.path.join("C:/Users/Liaqat Ali/Documents/ChiGaNet/NewHandPDImgs/HealthySpiral/HealthySpiral",img) for img in os.listdir("C:/Users/Liaqat Ali/Documents/ChiGaNet/NewHandPDImgs/HealthySpiral/HealthySpiral")]
X_PD_All = [os.path.join("C:/Users/Liaqat Ali/Documents/ChiGaNet/NewHandPDImgs/PatientSpiral/PatientSpiral",img) for img in os.listdir("C:/Users/Liaqat Ali/Documents/ChiGaNet/NewHandPDImgs/PatientSpiral/PatientSpiral")]

In [4]:
# load the VGG16 network
print("[INFO] loading network...")

# chop the top dense layers, include_top=False
model = VGG16(weights="imagenet", include_top=False)
#model.summary()

[INFO] loading network...


In [5]:
def create_features(dataset, pre_model):

    x_scratch = []

    # loop over the images
    for imagePath in dataset:

        # load the input image and image is resized to 224x224 pixels
        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)

        # preprocess the image by (1) expanding the dimensions and
        # (2) subtracting the mean RGB pixel intensity from the
        # ImageNet dataset
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)

        # add the image to the batch
        x_scratch.append(image)

    x = np.vstack(x_scratch)
    features = pre_model.predict(x, batch_size=32)
    features_flatten = features.reshape((features.shape[0], 7 * 7 * 512))
    return x, features, features_flatten

In [6]:
H_x_all, H_features_all, H_features_flatten_all = create_features(X_H_All, model)
print(H_x_all.shape)
print(H_features_all.shape)
print(H_features_flatten_all.shape)

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step
(140, 224, 224, 3)
(140, 7, 7, 512)
(140, 25088)


In [7]:
PD_x_all, PD_features_all, PD_features_flatten_all = create_features(X_PD_All, model)
print(PD_x_all.shape)
print(PD_features_all.shape)
print(PD_features_flatten_all.shape)

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step
(124, 224, 224, 3)
(124, 7, 7, 512)
(124, 25088)


In [8]:
X_H_all = H_features_flatten_all
X_PD_all = PD_features_flatten_all

print(X_PD_all.shape)
print(X_H_all.shape)

Y_PD_all = np.ones(124)
Y_H_all = np.zeros(140)
print(Y_PD_all.shape)
print(Y_H_all.shape)

(124, 25088)
(140, 25088)
(124,)
(140,)


In [9]:
Y_all = np.append(Y_PD_all, Y_H_all, axis=0)
print(Y_all.shape)
X_all = np.append(X_PD_all, X_H_all, axis=0)
print(X_all.shape)

(264,)
(264, 25088)


In [10]:
X_HandPD_New = X_all
Y_HandPD_New = Y_all
print(X_HandPD_New.shape)
print(Y_HandPD_New.shape)

(264, 25088)
(264,)


In [11]:
#Importaing Testing Data from HandPD
print(Y_HandPD_New)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:

X_H_All = [os.path.join("C:/Users/Liaqat Ali/Documents/ChiGaNet/HandPDImgs/Spiral_HandPD/SpiralControl",img) for img in os.listdir("C:/Users/Liaqat Ali/Documents/ChiGaNet/HandPDImgs/Spiral_HandPD/SpiralControl")]
X_PD_All = [os.path.join("C:/Users/Liaqat Ali/Documents/ChiGaNet/HandPDImgs/Spiral_HandPD/SpiralPatients",img) for img in os.listdir("C:/Users/Liaqat Ali/Documents/ChiGaNet/HandPDImgs/Spiral_HandPD/SpiralPatients")]

In [13]:
X_H_All = natsort.natsorted(X_H_All,reverse=False)
X_PD_All = natsort.natsorted(X_PD_All,reverse=False)

In [14]:
# load the VGG16 network
print("[INFO] loading network...")

# chop the top dense layers, include_top=False
model = VGG16(weights="imagenet", include_top=False)
#model.summary()

[INFO] loading network...


In [15]:
H_x_all, H_features_all, H_features_flatten_all = create_features(X_H_All, model)
print(H_x_all.shape)
print(H_features_all.shape)
print(H_features_flatten_all.shape)

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 970ms/step
(72, 224, 224, 3)
(72, 7, 7, 512)
(72, 25088)


In [16]:
PD_x_all, PD_features_all, PD_features_flatten_all = create_features(X_PD_All, model)
print(PD_x_all.shape)
print(PD_features_all.shape)
print(PD_features_flatten_all.shape)

10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step
(296, 224, 224, 3)
(296, 7, 7, 512)
(296, 25088)


In [17]:
X_H_all = H_features_flatten_all
X_PD_all = PD_features_flatten_all

print(X_PD_all.shape)
print(X_H_all.shape)

Y_PD_all = np.ones(296)
Y_H_all = np.zeros(72)
print(Y_PD_all.shape)
print(Y_H_all.shape)

(296, 25088)
(72, 25088)
(296,)
(72,)


In [18]:
Y_HandPD = np.append(Y_PD_all, Y_H_all, axis=0)
print(Y_HandPD.shape)
X_HandPD = np.append(X_PD_all, X_H_all, axis=0)
print(X_HandPD.shape)

(368,)
(368, 25088)


In [19]:
>>> from sklearn.svm import LinearSVC
>>> from sklearn.feature_selection import SelectFromModel

>>> model_FS = SelectKBest(chi2, k=130) # We tried different k i.e. 20, 30, 40, 50, ..., 150 and found out that 130 is optimal
>>> model_FS.fit(X_HandPD_New, Y_HandPD_New)
>>> X_FS_HandPD_New = model_FS.transform(X_HandPD_New)
>>> X_FS_HandPD = model_FS.transform(X_HandPD)
>>> print(X_FS_HandPD_New.shape)
>>> print(X_FS_HandPD.shape)

(264, 130)
(368, 130)


# **Developing Model under LOSO Cross Validation on the New HandPD Data**

**1. Time by GA**

In [20]:
opt_alpha = 0
opt_n1 = 0
opt_n2 = 0
    
clf = MLPClassifier( random_state=1)
x = range(1,31)
#param_grid= {'gamma': Continuous(0.0001, 1000, distribution='log-uniform'),'C': Continuous(0.0001, 1000, distribution='log-uniform')}
param_grid = {
    "hidden_layer_sizes": Categorical(list(itertools.combinations(x, 2))),
    "alpha": Categorical([1e-5]),
    "solver": Categorical(['lbfgs'])
}
cv = KFold(n_splits=66)

evolved_estimator = GASearchCV(estimator=clf,
                           cv=cv,
                           scoring='accuracy',
                           population_size=30,
                           generations=25,
                           tournament_size=3,
                           elitism=True,
                           crossover_probability=0.8,
                           mutation_probability=0.1,
                           param_grid=param_grid,
                           criteria='max',
                           algorithm='eaMuPlusLambda',
                           n_jobs=-1,
                           verbose=False,
                           keep_top_k=4)

# Record the start time
start_time = time.time()

evolved_estimator.fit(X_FS_HandPD_New, Y_HandPD_New)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken by the genetic algorithm for hyperparameter optimization: {elapsed_time:.2f} seconds")



Time taken by the genetic algorithm for hyperparameter optimization: 96.83 seconds


# Grid Search

In [21]:
# Time the Grid Search
# Define the range for hidden layer sizes (1 to 30 neurons for both layers)
layer_range = range(1, 31)
hidden_layer_combinations = list(itertools.product(layer_range, repeat=2))  # All combinations of two hidden layers

# Define the parameter grid for Grid Search
param_grid = {
    'alpha': [1e-5],  # Fix alpha to 1e-5
    'hidden_layer_sizes': hidden_layer_combinations,  # Search all combinations of hidden layer sizes
    'solver': ['lbfgs'],  # Fix solver to 'lbfgs'
}



# Initialize GridSearchCV with the updated param_grid (as a list)
grid_search = GridSearchCV(estimator=clf,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv,
                           n_jobs=-1,
                           verbose=0)
start_time = time.time()
# Fit the Grid Search model
grid_search.fit(X_FS_HandPD_New, Y_HandPD_New)

# Record end time for Grid Search
end_time = time.time()
grid_search_time = end_time - start_time
print(f"Time taken by Grid Search: {grid_search_time:.2f} seconds")



Time taken by Grid Search: 1202.12 seconds


# Random Search

In [22]:
# Time the Grid Search
# Define the range for hidden layer sizes (1 to 30 neurons for both layers)
layer_range = range(1, 31)
hidden_layer_combinations = list(itertools.product(layer_range, repeat=2))  # All combinations of two hidden layers

# Define the parameter grid for Grid Search
param_grid = {
    'alpha': [1e-5],  # Fix alpha to 1e-5
    'hidden_layer_sizes': hidden_layer_combinations,  # Search all combinations of hidden layer sizes
    'solver': ['lbfgs'],  # Fix solver to 'lbfgs'
}


random_search = RandomizedSearchCV(estimator=clf,
                                   param_distributions=param_grid,
                                   n_iter=450,  # Number of random combinations to sample (change as needed)
                                   scoring='accuracy',
                                   cv=cv,
                                   n_jobs=-1,  # Use all available CPU cores
                                   verbose=1,  # Print progress
                                   random_state=42)  # Ensure reproducibility

# Record the start time
start_time = time.time()

# Fit the model using RandomizedSearchCV
random_search.fit(X_FS_HandPD_New, Y_HandPD_New)

# Record the end time
end_time = time.time()

# Calculate the time taken for Randomized Search
random_search_time = end_time - start_time
print(f"Time taken by RandomizedSearchCV: {random_search_time:.2f} seconds")


Fitting 66 folds for each of 450 candidates, totalling 29700 fits
Time taken by RandomizedSearchCV: 520.22 seconds


In [23]:
print("Done")

Done
